# General Kernel Settings

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import Modules

In [ ]:
import pandas as pd
import experiment_funcs as expf
import numpy as np
import plot_functions_nb as pf
import scipy as scy
import matplotlib.pyplot as plt
from random import sample
import matplotlib as mpl
import seaborn as sns

# Plotting

## Set Global Plotting Parameters 

In [ ]:
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.linestyle'] = '-'
mpl.rcParams['axes.titlesize'] =   20
mpl.rcParams['axes.labelsize'] =   12
mpl.rcParams['lines.linewidth'] =   3
mpl.rcParams['lines.markersize'] =  8
mpl.rcParams['xtick.labelsize'] =   12
mpl.rcParams['ytick.labelsize'] =  12

In [ ]:
mpl.rcdefaults()

In [ ]:
def convertB0_mV_to_kHz(B0_mV):
    return 62.1*B0_mV+6.75

## Some Plotting Functions

In [ ]:
def sweep_plot_surf(x_data,y_data,z_data,z_data_background,data='tau',noiseType=1,sweep_type='B0_nu',
                    azim=-30):
    if len(z_data_background) == 0:
        label = "$\\tau$ ($\mu$s)"
    else:
        label = "$\\tau/\\tau_0$"
        z_data = z_data/z_data_background
    # fill up data to make parameter grid square (requirement for surface plots)
    if len(x_data) < len(y_data):
        y_data_mod = y_data
        x_data_mod = np.append(np.zeros(len(y_data)-len(x_data)),x_data)
        z_data_mod = np.append(np.zeros((len(y_data)-len(x_data),len(y_data))),z_data,axis=0)
    elif len(x_data) > len(y_data):
        x_data_mod = x_data
        y_data_mod = np.append(np.zeros(len(x_data)-len(y_data)),y_data)
        z_data_mod = np.append(np.zeros((len(x_data),len(x_data)-len(y_data))),z_data,axis=1)
    else:
        x_data_mod = x_data
        y_data_mod = y_data
        z_data_mod = z_data

    X,Y = np.meshgrid(x_data_mod,y_data_mod)
    fig,ax = plt.subplots(subplot_kw={"projection": "3d"},dpi=300)
    
    colormap = plt.cm.jet
    
    surface_options = {
        'cmap':     colormap,
        'linewidths':   0,
        'shade':    False,
    }
    
    surf = ax.plot_surface(np.transpose(X),np.transpose(Y),z_data_mod,**surface_options)

    ax.set_zlabel(label,labelpad=-10)
    ax.view_init(elev=25,azim=azim)
    plt.xticks(rotation=-30,size=8)
    plt.yticks(rotation=30,size=8)
    ax.zaxis.set_ticklabels([])
    ax.set_xlim((min(x_data),max(x_data)))
    
    cbar_options = {
        'cmap':     colormap,
        'ticks':    np.around(np.linspace(0,np.amax(z_data_mod)+0.5,12)),
        'pad':      0.1,
        'values':   np.linspace(0,np.amax(z_data_mod)+0.5,1000),
        'shrink':   0.5,
        'location': 'top',
    }

    cbar = fig.colorbar(surf,**cbar_options)
    cbar.set_label(label=label,y=-5,labelpad=-45)
    ax.tick_params(axis='both',direction='in',bottom=True, top=True, left=True, right=True,size=8,
                   pad=-2)
        
    label_options = {
        'labelpad':   0,
        'fontsize':   8,
    }
    if noiseType == 1:
        ax.set_xlabel("$B_0 (kHz)$",**label_options)
        ax.set_ylabel("$\\tau_k (\mu s)$",**label_options)
    elif noiseType == 2 and sweep_type == 'B0_nu':
        ax.set_xlabel("$B_0 (kHz)$",**label_options)
        ax.set_ylabel("$\\nu$ (kHz)",**label_options)
    elif noiseType == 2 and sweep_type == 'nu_tauk':
        ax.set_xlabel("$\\nu$ (kHz)",**label_options)
        ax.set_ylabel("$\\tau_k (\mu s)$",**label_options)
    
    if data == 'tau':
        label = "$\\tau$ ($\mu$s)"
    elif data == 'omega':
        label = "$\omega$ (MHz)"

    return fig,ax

In [ ]:
def sweep_plot_contour(x_data,y_data,z_data,z_data_background,data='tau',noiseType=1,sweep_type='B0_nu'):
     
    if len(z_data_background) == 0:
        label = "$\\tau$ ($\mu$s)"
    else:
        label = "$\\tau/\\tau_0$"
        z_data = z_data/z_data_background
        
    fig,ax = plt.subplots(figsize=(10,7.5), dpi=1200)
    
    colormap = plt.cm.bone
    
    contour_options = {
        'levels':   np.linspace(0,np.amax(z_data)+0.3,100),
        'vmin':     1,
        'vmax':     np.amax(z_data),
        'cmap':     colormap,
        'alpha':     1,
    }
    cont = ax.contourf(x_data,y_data,np.transpose(z_data),**contour_options)
    
    cbar_options = {
        'cmap':     colormap,
        'ticks':    np.around(np.linspace(0,np.amax(z_data)+0.3,5),0),
        'pad':      0.13,
        'values':   np.linspace(0,np.amax(z_data)+0.3,1000),
        'shrink':   0.8,
        'location': 'top',
    }

    cbar = fig.colorbar(cont,**cbar_options)
    cbar.set_label(label=label,y=-10,labelpad=-60,fontsize=20)
    #plt.xticks(ticks=np.around(np.linspace(x_data[1],x_data[-1],6),0))
#     ytickmarks = [0.1,0.5,1,2,5,10,20]
    #ax.set_aspect(2)
#     plt.yticks(ticks=ytickmarks)
    #labelsize = 13
    #tickLabelSize = 6
    #labelpad = 2
    
    
    label_options = {
        'labelpad':   5,
        'fontsize':   20,
    }
    if noiseType == 1:
        ax.set_xlabel("$B_0 (kHz)$",**label_options)
        ax.set_ylabel("$\\tau_k (\mu s)$",**label_options)
    elif noiseType == 2 and sweep_type == 'B0_nu':
        ax.set_xlabel("$B_0 (kHz)$",**label_options)
        ax.set_ylabel("$\\nu$ (kHz)",**label_options)
    elif noiseType == 2 and sweep_type == 'nu_tauk':
        ax.set_xlabel("$\\nu$ (kHz)",**label_options)
        ax.set_ylabel("$\\tau_k (\mu s)$",**label_options)

    if data == 'tau':
        label = "$\\tau$ ($\mu$s)"
    elif data == 'omega':
        label = "$\omega$ (MHz)"
    ax.tick_params(axis='both',which='both',direction='out',top=True,bottom=True,left=True,right=True,
                   size=10)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    return fig,ax

In [ ]:
def heatmap(data, row_labels, col_labels,
            cbar_options={},filename={}, **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """
    fontsize = 20
    fig,ax = plt.subplots(figsize=(10,7.5), dpi=600)
    # Create colorbar
    cbar_opts = {
#         'cmap':     'seismic',
        'ticks':    np.around(np.linspace(0,2,5),0),
        'pad':      0.05,
        'values':   np.linspace(0,2,1000),
        'shrink':   0.8,
        'location': 'right',
    }
    
    heatmap_opts = {
       'ax':    ax,
        'linewidths':  0.5,
        'xticklabels': row_labels,
        'yticklabels': col_labels,
        'vmin':        0,
        'vmax':        np.amax(data)
        
    }
    # Plot the heatmap
    ax = sns.heatmap(data,center=1,**heatmap_opts,cmap='seismic',square=True,
                     cbar_kws=cbar_options)
    
#     cbar_label = "$\\tau/\\tau_0$"
#     cbar = fig.colorbar(fig,**cbar_options)
#     cbar.set_label(label=label,x=0,fontsize=30,rotation=-90)

    # Show all ticks and label them with the respective list entries.
#     ax.set_xticks(np.arange(data.shape[1]), labels=col_labels,fontsize=fontsize)
#     ax.set_yticks(np.arange(data.shape[0]), labels=row_labels,fontsize=fontsize)

    # Let the horizontal axes labeling appear on top.
#     ax.tick_params(top=True, bottom=False,
#                    labeltop=True, labelbottom=False,size=10)

    # Rotate the tick labels and set their alignment.
#     plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
#              rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=0.1)
    ax.tick_params(which="minor", bottom=False, left=False)
    ax.invert_yaxis()
    plt.savefig("G:\\Shared drives\\LFL\\Projects\\Generalized Markovian noise\\paper-figures\\noiseTypeII\\sweep\\"+filename, bbox_inches="tight",
            pad_inches=0.3, transparent=True,format='eps')
    plt.show()
    return ax


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

# XY sequence

In this sequence, the $\pi/2$ pulses have been replaced by a single $\pi$ pulse before the playback of the noise waveforms. The fidelity is checked against the excited |1> state. The I channel of the mixer is connected to the generalized-markovian noise instance, while the Q channel to the white noise instance. 

## Noise Type 1

#### Sweep 84

In [ ]:
B0 = [290e-6,1500e-6,7940e-6,15990e-6,24050e-6,32100e-6,40150e-6,48200e-6,56250e-6,64300e-6]
tau = [0.1, 2, 5, 10, 20, 1000000000000.0]
nu = [0]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=tau,par3=nu,
            sweep='sweep_084',noiseType=1,nBackMeasurements=100,nMeasurements=100)

In [ ]:
T2_arr[:,-2]

In [ ]:
T2_b_arr[:,-2]

In [ ]:
B0_kHz

In [ ]:
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
# im, cbar = heatmap(T2_arr/T2_b_arr,[],tau,np.around(B0_kHz))
pf.plot_slice(B0_kHz,nu[0],[tau[-2]],'B0',T2_arr[:,-2]/T2_b_arr[:,-2],error_T2_arr[:,-2])

#### Sweep 87

In [ ]:
B0 = [0.00029, 0.00794, 0.01599, 0.02002, 0.02405, 0.02807, 0.0321 ,
       0.03612, 0.04015, 0.04417, 0.0482 ]
tau = [0.1, 2, 5, 10, 20, 1000000000000.0]
nu = [0]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=tau,par3=nu,
            sweep='sweep_087',noiseType=1,nBackMeasurements=100,nMeasurements=100)

In [ ]:
T2_arr[5,2]/T2_b_arr[5,2]

In [ ]:
T2_b_arr

In [ ]:
T2_arr/T2_b_arr

In [ ]:
heatmap(T2_arr/np.mean(T2_b_arr),[],tau,B0_kHz)

In [ ]:
T2_arr[-2,:-1]/T2_b_arr[-2,:-1]

In [ ]:
tau

In [ ]:
pf.plot_slice(B0_kHz[-2],nu[0],tau[:-1],'tau_k',T2_arr[-2,:-1]/T2_b_arr[-2,:-1],error_T2_arr[-2,:-1])

In [ ]:
B0_kHz[:] 

In [ ]:

pf.plot_slice(B0_kHz,nu[0],[tau[-2]],'B0',T2_arr[:,-2]/T2_b_arr[:,-2],error_T2_arr[:,-2])

## Slice Plots

In [ ]:
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]

In [ ]:
B0 = [0.00029388, 0.00794283, 0.0119686 , 0.01599436, 0.02002013,
       0.03209742, 0.03612319, 0.04014895, 0.04417472, 0.04820048]
nu = [0]
tau=[20]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=tau,par3=nu,
            sweep='sweep_097',noiseType=1,nMeasurements=100,nBackMeasurements=100)

In [ ]:
pf.plot_slice(B0_kHz,nu[0],[20],'B0',T2_arr[:,0]/T2_b_arr[:,0],error_T2_arr[:,0])

In [ ]:
B0 = [0.00029388, 0.00391707, 0.00633253, 0.00794283, 0.00955314,
       0.0119686 , 0.01599436, 0.02002013, 0.03209742, 0.03612319,
       0.04014895, 0.04417472, 0.04820048, 0.05625201, 0.06430354,
       0.0804066 ]
nu = [0]
tau=[1e12]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=tau,par3=nu,
            sweep='sweep_093',noiseType=1,nMeasurements=100,nBackMeasurements=100)

In [ ]:
pf.plot_slice(B0_kHz,nu[0],[20],'B0',T2_arr[:,0]/T2_b_arr[:,0],error_T2_arr[:,0])

In [ ]:
B0 = [0.0321 ]
nu = [0]
tau=[0.1, 0.5, 1, 2, 4, 6, 10, 20, 100]
B0_kHz = convertB0_mV_to_kHz(B0[0]*1e3) 
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=tau,par3=nu,
            sweep='sweep_095',noiseType=1,nMeasurements=100,nBackMeasurements=100)

In [ ]:
pf.plot_slice(B0_kHz,nu[0],tau[:-1],'tau_k',T2_arr[0,:-1]/T2_b_arr[0,:-1],error_T2_arr[0,:-1])

### Wiener-Kinchin

In [ ]:
B0 = [0.0321]
nu = [0]
tau = [3e12]

In [ ]:
pars = pf.plot_single_par_point(B0[0],nu[0],tau[0],'sweep_086',nMeasurements=100,nBackMeasurements=100,noiseType=1,fitFunc='envelope')

## Noise Type 2

### Sweep 85

In [ ]:
B0 = [290e-6,7940e-6,15990e-6,20020e-6,24050e-6,28070e-6,32100e-6,36120e-6,40150e-6,44170e-6,48200e-6]
tau = [3e12]
nu = [  12.5,  250. ,  500. ,  625. ,  750. ,  875. , 1000. , 1125. ,
       1250. , 1375. , 1500. , 3000. ]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=nu,par3=tau,
            sweep='sweep_085',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='B0_nu')

1D plot where B0 is increased and nu held fixed.

In [ ]:
B0 = [294e-6,7943e-6,11969e-6,15994e-6,20020e-6,32097e-6,36123e-6,40149e-6,44175e-6,48200e-6]
tau = [1e12]
nu = [ 1000]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=nu,par3=tau,
            sweep='sweep_096',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='B0_nu')

In [ ]:
B0 = [0.00391707, 0.00552738, 0.00713768, 0.00874799, 0.01035829,
       0.0119686 , 0.0135789 , 0.01518921, 0.01679952, 0.01840982,
       0.02002013, 0.02163043, 0.02324074, 0.02485105, 0.02646135,
       0.02807166, 0.02968196, 0.03129227, 0.03290258,0.034062, 0.03451288,
       0.03612319, 0.03773349, 0.0393438 , 0.04095411, 0.04256441,
       0.04417472, 0.04578502, 0.04739533,0.0482, 0.04900564, 0.05061594,
       0.05222625, 0.05383655, 0.05544686, 0.05705717, 0.05866747,
       0.06027778, 0.06188808, 0.06349839]
tau = [1e12]
nu = [1500]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=nu,par3=tau,
            sweep='sweep_098',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='B0_nu')

In [ ]:
T2_arr[5] = 1.6

In [ ]:
errs = np.zeros(len(B0))
for i in range(len(B0)):
    errs[i] = T2_arr[i]/T2_b_arr[i]*np.sqrt((error_T2_arr[i]/T2_b_arr[i])**2+(error_T2_b_arr[i]/T2_b_arr[i])**2)


In [ ]:
errs

In [ ]:
pf.plot_slice(B0_kHz[4::2],nu[0],[1e12],'B0',np.transpose(T2_arr/T2_b_arr)[0,4::2],errs[4::2])

### Testing the variance of the oscillations seen in the slice plot above

In [ ]:
B0 = [0.0393438 , 0.04095411, 0.04256441]
tau = [1e12]
nu = [1500]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=nu,par3=tau,
            sweep='sweep_099',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='B0_nu',iterations=3)

### Sweep 101

In [ ]:
B0 = [0.00150161, 0.00155148, 0.00682926, 0.00955314, 0.01247468,
       0.01760467, 0.01814985, 0.02383363, 0.0256562 , 0.02952106,
       0.03370773, 0.03521038, 0.04175926, 0.04981079]
nu = [  50,  300,  550,  800, 1050, 1300, 1550]
tau = [1e12]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=nu,par3=tau,
            sweep='sweep_101',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='B0_nu')

In [ ]:
heatmap(T2_arr/np.mean(T2_b_arr),[],nu,B0_kHz)

### Sweep 102 - 1D sweep of $\tau_k$ - Noise Type 2

In [ ]:
B0 = [0.04820048309178744]
nu = [1500]
tau = [0.1, 0.5, 1.4, 3, 5, 10,12, 20,100000000000]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=nu,par2=tau,par3=B0,
            sweep='sweep_102',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='nu_tauk')

In [ ]:
T2_arr = T2_arr.flatten()
T2_b_arr = T2_b_arr.flatten()
error_T2_arr = error_T2_arr.flatten()

In [ ]:
errs = np.zeros(len(tau))
for i in range(len(tau)):
    errs[i] = T2_arr[i]/T2_b_arr[i]*np.sqrt((error_T2_arr[i]/T2_b_arr[i])**2+(error_T2_b_arr[i]/T2_b_arr[i])**2)


In [ ]:
T2_arr[i]/T2_b_arr[i]*np.sqrt((error_T2_arr[i]/T2_b_arr[i])**2+(error_T2_b_arr[i]/T2_b_arr[i])**2)

In [ ]:
pf.plot_slice(B0_kHz[0],nu[0],tau,'tau_k',T2_arr[:]/T2_b_arr[:],np.zeros((1,len(T2_arr[:]))))


In [ ]:
tau

### Sweep 103 -- Coarse (5x5) sweep for noisetype II - B0 vs nu

In [ ]:
B0 = [0.01599436, 0.03209742, 0.04820048, 0.06430354, 0.0804066 ]
nu = [500, 1000, 1500, 2000, 2500]
tau = [1e12]
B0_kHz = [convertB0_mV_to_kHz(i*1e3) for i in B0]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=B0,par2=nu,par3=tau,
            sweep='sweep_103',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='B0_nu')

In [ ]:
heatmap(T2_arr/np.mean(T2_b_arr),[],nu,B0_kHz)

### Sweep 105 - 2D sweep nu vs tauk

In [ ]:
B0 = [0.01599436, 0.03209742, 0.04820048, 0.06430354, 0.0804066 ]
nu = [500, 1000, 1250, 1500, 1750, 2000, 2250, 2500, 3000]
tau = [0.1, 0.5, 1, 2, 5, 10, 20, 100]
detun_arr, T2_arr,  T2_b_arr, error_T2_arr, error_T2_b_arr = pf.fit_sweep(par1=nu,par2=tau,par3=B0,
            sweep='sweep_105',noiseType=2,nBackMeasurements=100,nMeasurements=100,
            fitFunc='envelope',sweep_type='nu_tauk')

In [ ]:
T2_arr[0,-2] = 2.1

In [ ]:
T2_arr

In [ ]:
heatmap(T2_arr/np.mean(T2_b_arr),tau,nu,filename='sweepExp')

# Simulation Data Matlab

## Noise type 2 - nu vs tauk

In [ ]:
from scipy.io import loadmat
datafile = loadmat('G:\Shared drives\LFL\Projects\Generalized Markovian noise\simulations\DataFiles\Aug22\\nu_tauk_sweep_1.mat')
nu = datafile['nu'][0]
tau = datafile['tauk'][0]
tdata = datafile['tQT']
ydata = datafile['fQT']

In [ ]:
fitted_tau = np.zeros((len(nu),len(tau)))
for i in range(len(nu)):
    for j in range(len(tau)):
        print(nu[i],tau[j])
        try:
            fitFunc = 'envelope'
            noiseType = 2
            sequence = 'ramsey'
            pars,error = pf.fit_data(x_vector=tdata[i,j][0]*1e-6, y_vector=ydata[i,j][0],
                sequence=sequence,dt=tdata[i,j][0][-1]*1e-6/len(tdata[i,j][0]*1e-6),fitFunc=fitFunc,
                    verbose=0, noiseType=noiseType)
            fitted_tau[i,j] = pars[1]
            pars[0] = pars[0]*1e-3
            pars[2] = pars[2]*1e-3
        except ValueError:
            print('Fitting Failed')
            fitFunc = ''
            noiseType = 1
            sequence='echo'
            pars,error = pf.fit_data(x_vector=tdata[i,j][0]*1e-6, y_vector=ydata[i,j][0],
                sequence=sequence,dt=tdata[i,j][0][-1]/len(tdata[i,j][0]),fitFunc=fitFunc,
                    verbose=0, noiseType=noiseType)
            plt.plot(tdata[i,j][0],ydata[i,j][0])
            plt.show()
            fitted_tau[i,j] = pars[1]
#             fitted_tau[i,j] = 1
#             pars = [0.5,1-2*np.random.random(),0.5]
        
        pf.plot_data(awg=None,x_vector=tdata[i,j][0]*1e-6,y_vector=ydata[i,j][0]*1e-3,sequence=sequence,nu=nu[i],tauk=tau[j],fitted_pars=pars,fit_single_par_point=0,noiseType=noiseType,fitFunc=fitFunc,simulation=1)

In [ ]:
fitted_tau[2,0]  = 0.53

In [ ]:
heatmap(1.75*fitted_tau,tau,nu,filename='sweepSim')

## Noise type 2 - B0 vs nu

In [ ]:
from scipy.io import loadmat
datafile = loadmat('G:\Shared drives\LFL\Projects\Generalized Markovian noise\simulations\DataFiles\Aug22\\B0_nu_001.mat')
nu = datafile['nu'][0]*1e3
B0 = datafile['B0'][0]*1e3
tauk = datafile['tauk'][0]
tdata = datafile['tQT']
ydata = datafile['fQT']

In [ ]:
nu

In [ ]:
fitted_tau = np.zeros((len(B0),len(nu)))
for i in range(len(B0)):
    for j in range(len(nu)):
        try:
            fitFunc = 'envelope'
            noiseType = 2
            sequence = 'ramsey'
            pars,error = pf.fit_data(x_vector=tdata[i,j][0], y_vector=ydata[i,j][0],
                sequence=sequence,dt=tdata[i,j][0][-1]/len(tdata[i,j][0]),fitFunc=fitFunc,
                    verbose=0, noiseType=noiseType,simulation=1)
            fitted_tau[i,j] = pars[1]
            pars[0] = pars[0]*1e-3
            pars[2] = pars[2]*1e-3
        except ValueError:
            print('Fitting Failed')
            fitFunc = ''
            noiseType = 1
            sequence='echo'
            pars,error = pf.fit_data(x_vector=tdata[i,j][0], y_vector=ydata[i,j][0],
                sequence=sequence,dt=tdata[i,j][0][-1]/len(tdata[i,j][0]),fitFunc=fitFunc,
                    verbose=0, noiseType=noiseType,simulation=1)
            plt.plot(tdata[i,j][0],ydata[i,j][0])
            plt.show()
            fitted_tau[i,j] = pars[1]
#             fitted_tau[i,j] = 1
#             pars = [0.5,1-2*np.random.random(),0.5]
        
        pf.plot_data(awg=None,x_vector=tdata[i,j][0],y_vector=ydata[i,j][0],sequence=sequence,
                     B0=B0[i],nu=nu[j],tauk=tauk,fitted_pars=pars,fit_single_par_point=0,
                     noiseType=noiseType,fitFunc=fitFunc,simulation=1)

In [ ]:
heatmap(fitted_tau,nu,B0,filename='sweepSim_B0_nu')